In [1]:
# 1. Imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
print(torch.backends.mps.is_available())   # should be True on M-series


/Users/jmsung/miniconda3/envs/bio-ml/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True


In [2]:
# 2. Load Classification Dataset
classification_df = pd.read_csv("../data/classification_data.csv")
classification_df.head()

,id,text,label
0,1,DrugZ caused severe rashes in some participants.,Adverse Effect
1,2,No significant side effects were observed duri...,Positive Outcome
2,3,Participants were monitored every two weeks.,Neutral Observation
3,4,Increased liver enzymes were noted post-treatm...,Adverse Effect
4,5,Patients were instructed to maintain a food di...,Neutral Observation


In [3]:
classification_df['label'].value_counts().sort_index()

label
Adverse Effect         352
Neutral Observation    335
Positive Outcome       313
Name: count, dtype: int64

In [4]:
# 3. Encode Labels
label_encoder = LabelEncoder()
classification_df['label_encoded'] = label_encoder.fit_transform(classification_df['label'])
classification_df.head()

,id,text,label,label_encoded
0,1,DrugZ caused severe rashes in some participants.,Adverse Effect,0
1,2,No significant side effects were observed duri...,Positive Outcome,2
2,3,Participants were monitored every two weeks.,Neutral Observation,1
3,4,Increased liver enzymes were noted post-treatm...,Adverse Effect,0
4,5,Patients were instructed to maintain a food di...,Neutral Observation,1


In [5]:
# 4. Train-Test Split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    classification_df['text'].tolist(),
    classification_df['label_encoded'].tolist(),
    test_size=0.2,
    random_state=42
)

for i in range(5):
    print(f"Train Text: {train_texts[i]}")
    print(f"Train Label: {train_labels[i]}")
    print()

Train Text: No significant side effects were observed during the trial.
Train Label: 2

Train Text: Participants showed improved lung function after therapy with DrugC.
Train Label: 2

Train Text: Severe allergic reactions were observed following DrugB administration.
Train Label: 0

Train Text: Patients were instructed to maintain a food diary.
Train Label: 1

Train Text: Participants were monitored every two weeks.
Train Label: 1



In [6]:
# 5. Tokenization
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [7]:
# 6. Dataset Class
torch.backends.cuda.matmul.allow_tf32 = True

class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ClassificationDataset(train_encodings, train_labels)
val_dataset = ClassificationDataset(val_encodings, val_labels)

In [8]:
# 7. Model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(label_encoder.classes_))

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# 8. Training Arguments
training_args = TrainingArguments(
    output_dir="/tmp/results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="/tmp/logs",
    logging_steps=10,
    save_strategy="no",
    load_best_model_at_end=False,
)

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'